# Implement an LCEL with a memory


## Import modules


In [4]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

## Implement Custom LCEL and Functions about memories


In [5]:
def add_message(input: str, output: str) -> None:
    memory.save_context({"input": input}, {"output": output})


def load_memory(_) -> str:
    return memory.load_memory_variables({})["history"]


def invoke_chain(question: str) -> None:
    result = emoji_chain.invoke({"movie": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)


emoji_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You must answer with three emoji to movie name"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}"),
    ]
)
llm = ChatOpenAI(temperature=0.1)
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

emoji_chain = RunnablePassthrough.assign(history=load_memory) | emoji_prompt | llm

## Generate Completion


In [6]:
invoke_chain("Harry Potter")
invoke_chain("Parasite")

⚡️🧙‍♂️🔮
🏠👨‍👩‍👧‍👦🔪


## Validate Memory Use


In [7]:
title_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI that tracks the user's conversation history"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
title_chain = RunnablePassthrough.assign(history=load_memory) | title_prompt | llm
completion = title_chain.invoke({"question": "What movies did I ask about?"})
completion.content

'You asked about "Harry Potter" and "Parasite."'